In [1]:
!git clone https://github.com/yl4579/StyleTTS-VC.git

Cloning into 'StyleTTS-VC'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 85 (delta 12), reused 6 (delta 0), pack-reused 52
Receiving objects: 100% (85/85), 102.59 MiB | 1.42 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Updating files: 100% (44/44), done.


In [2]:
%cd StyleTTS-VC/

/Users/jordanwilke/i3_speech_emotion_recognition/StyleTTS-VC


In [3]:
!pip install SoundFile torchaudio munch torch pydub pyyaml librosa git+https://github.com/resemble-ai/monotonic_align.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/resemble-ai/monotonic_align.git to /private/var/folders/gc/sf7fc2ys73768x4y8_p7lj_80000gn/T/pip-req-build-wny2mq6o
  Running command git clone --filter=blob:none --quiet https://github.com/resemble-ai/monotonic_align.git /private/var/folders/gc/sf7fc2ys73768x4y8_p7lj_80000gn/T/pip-req-build-wny2mq6o
  Resolved https://github.com/resemble-ai/monotonic_align.git to commit 78b985be210a03d08bc3acc01c4df0442105366f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for munch from https://files.pythonhosted.org/packages/56/b3/7c69b37f03260a061883bec0e7b05be7117c1b1c85f5212c72c8c2bc3c8c/munch-4.0.0-py2.py3-none-any.whl.metadata
  Created wheel for monotonic-align: filename=monotonic_align-1.2-cp39-cp39-macosx_10_9_x86_64.whl size=314097 sha256=ca7f9e145b5c8c7b772b1a1c84287276

In [5]:
!wget -O model.zip "https://drive.google.com/u/0/uc?id=1bJbj3alOSu51riHUQl4G1GOjlzulyg6M&export=download&confirm=1"
!unzip model.zip
%rm model.zip
!wget -O vocoder.zip "https://drive.google.com/u/0/uc?id=1RDxYknrzncGzusYeVeDo38ErNdczzbik&export=download&confirm=1"
!unzip vocoder.zip
%rm vocoder.zip

--2023-09-08 16:07:16--  https://drive.google.com/u/0/uc?id=1bJbj3alOSu51riHUQl4G1GOjlzulyg6M&export=download&confirm=1
Resolving drive.google.com (drive.google.com)... 142.250.217.142
Connecting to drive.google.com (drive.google.com)|142.250.217.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1bJbj3alOSu51riHUQl4G1GOjlzulyg6M&export=download&confirm=1 [following]
--2023-09-08 16:07:16--  https://drive.google.com/uc?id=1bJbj3alOSu51riHUQl4G1GOjlzulyg6M&export=download&confirm=1
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-as-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/anifq4hgqd797649rgt0ri3h51n2gulp/1694214375000/17287530179503835962/*/1bJbj3alOSu51riHUQl4G1GOjlzulyg6M?e=download&uuid=6969504e-46f2-4f13-aec6-976950aaf9eb [following]
--2023-09-08 16:07:16--  https://doc-0c-as-docs.googleusercontent.com/doc

#Start of inference

In [ ]:
!pip install datasets

In [6]:


# load packages
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa

from models import *
from utils import *

%matplotlib inline



In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'



to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(ref_dicts):
    reference_embeddings = {}
    for key, path in ref_dicts.items():
        wave, sr = librosa.load(path, sr=24000)
        audio, index = librosa.effects.trim(wave, top_db=25)
        if sr != 24000:
            audio = librosa.resample(audio, sr, 24000)
        mel_tensor = preprocess(audio).to(device)

        with torch.no_grad():
            ref = model.style_encoder(mel_tensor.unsqueeze(1))
        reference_embeddings[key] = (ref.squeeze(1), audio)

    return reference_embeddings



In [8]:
!pip install addict

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [9]:
# load hifi-gan

import sys
sys.path.insert(0, "./Demo/hifi-gan")

import glob
import os
import argparse
import json
import torch
from scipy.io.wavfile import write
from addict import Dict
from vocoder import Generator
import librosa
import numpy as np
import torchaudio

h = None

def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device)
    print("Complete.")
    return checkpoint_dict

def scan_checkpoint(cp_dir, prefix):
    pattern = os.path.join(cp_dir, prefix + '*')
    cp_list = glob.glob(pattern)
    if len(cp_list) == 0:
        return ''
    return sorted(cp_list)[-1]

cp_g = scan_checkpoint("Vocoder/LibriTTS/", 'g_')

config_file = os.path.join(os.path.split(cp_g)[0], 'config.json')
with open(config_file) as f:
    data = f.read()
json_config = json.loads(data)
h = Dict(json_config)

device = torch.device(device)
generator = Generator(h).to(device)

state_dict_g = load_checkpoint(cp_g, device)
generator.load_state_dict(state_dict_g['generator'])
generator.eval()
generator.remove_weight_norm()

Loading 'Vocoder/LibriTTS/g_00935000'
Complete.
Removing weight norm...


In [10]:


# load StyleTTS
model_path = "./Models/VCTK/epoch_2nd_00100.pth"
model_config_path = "./Models/VCTK/config.yml"

config = yaml.safe_load(open(model_config_path))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

model = build_model(Munch(config['model_params']), text_aligner, pitch_extractor)

params = torch.load(model_path, map_location='cpu')
params = params['net']
for key in model:
    if key in params:
        if not "discriminator" in key:
            print('%s loaded' % key)
            model[key].load_state_dict(params[key])
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]



mel_encoder loaded
linear_proj loaded
decoder loaded
pitch_extractor loaded
text_encoder loaded
style_encoder loaded
text_aligner loaded


In [18]:
%pwd
%cd ..

/Users/jordanwilke/i3_speech_emotion_recognition


In [19]:
import argparse, sys
import torchaudio
import torch
import os
from tqdm import tqdm
from TTS.api import TTS
import random
from datasets import Dataset, load_dataset, concatenate_datasets

try:
  dataset = load_dataset("mozilla-foundation/common_voice_13_0", "it")
except RuntimeError as e:
  print(e)
  !huggingface-cli login
  dataset = load_dataset("mozilla-foundation/common_voice_13_0", "it")

setup = 'long_sentences.csv' in os.listdir('..')
if setup:
  data = concatenate_datasets([dataset['train'],dataset['validation']])
  sentences = data.filter(lambda x: len(x['sentence'].split(' ')) > 25)

  sentences.to_csv('../long_sentences.csv', sep="\t")

data = load_dataset('csv', delimiter='\t', data_files='../long_sentences.csv', split='train')
male_data = data.filter(lambda x: x['gender']=='male')
female_data = data.filter(lambda x: x['gender']=='female')
#male_data = load_dataset('csv', delimiter='\t',data_files='long_male_sentences.csv', split='train')
#female_data = load_dataset('csv', delimiter='\t',data_files='long_female_sentences.csv', split='train')
print(male_data)
print(female_data)
all_data = concatenate_datasets([male_data, female_data])

Creating CSV from Arrow format: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 73.78ba/s]


Extracting data files: 100%|████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 726.16it/s]


Dataset csv downloaded and prepared to /Users/jordanwilke/.cache/huggingface/datasets/csv/default-d4683e5f39f01210/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 120
})
Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 11
})


In [20]:
output_path = './data/audio4analysis_vc_real'
input_path = './data/audio4analysis'
gender = None

In [21]:
if gender == 'male' or gender == 'm':
  speaker_ind = random.randrange(len(male_data))
  speaker = male_data[speaker_ind]
elif gender == 'female' or gender =='f':
  speaker_ind = random.randrange(len(female_data))
  speaker = female_data[speaker_ind]
else:
  speaker_ind = random.randrange(len(all_data))
  speaker = all_data[speaker_ind]

if not os.path.exists(output_path):
  os.makedirs(output_path)

In [22]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
!apt-get install ffmpeg

In [27]:

from pydub import AudioSegment
# get first 3 test sample as references

# test_path = val_path.replace('/val_list.txt', '/test_list.txt')
# _, test_list = get_data_path_list(train_path, test_path)

#Different from StarGAN as this doesn't have a number associated with each speaker in the ref_dicsts
ref_dicts = {}
# for j in range(3):
#     filename = test_list[j].split('|')[0]
#     name = filename.split('/')[-1].replace('.wav', '')
#     ref_dicts[name] = filename

f = speaker['path']

sound = AudioSegment.from_mp3(f)
sound.export('temp.wav', format="wav")

key = os.path.basename('temp.wav').replace('.wav', '')

ref_dicts[key] = 'temp.wav'

print(ref_dicts)

reference_embeddings = compute_style(ref_dicts)



{'temp': 'temp.wav'}


In [32]:
# src_path = 'Data/src/'
# converted_dicts = []
# for f in os.listdir(src_path):
#   # get last test sample as input
#   # filename = test_list[-1].split('|')[0]
#   filename = os.path.join(src_path, f)
#   audio, source_sr = librosa.load(filename, sr=24000)
#   audio, index = librosa.effects.trim(audio, top_db=30)
#   audio = audio / np.max(np.abs(audio))
#   audio.dtype = np.float32
#   source = preprocess(audio).to(device)
#   converted = conversion(source, filename, reference_embeddings)
#   converted_dicts.append((converted, audio))



In [28]:


def conversion(source, source_name, reference_embeddings):
  converted_samples = {}
  with torch.no_grad():
      mel_input_length = torch.LongTensor([source.shape[-1]])
      asr = model.mel_encoder(source)
      F0_real, _, F0 = model.pitch_extractor(source.unsqueeze(1))
      real_norm = log_norm(source.unsqueeze(1)).squeeze(1)

      for key, (ref, _) in reference_embeddings.items():
          out = model.decoder(asr, F0_real.unsqueeze(0), real_norm, ref.squeeze(1))

          c = out.squeeze()
          y_g_hat = generator(c.unsqueeze(0))
          y_out = y_g_hat.squeeze()

          converted_samples[source_name] = y_out.cpu().numpy()
      return converted_samples



In [45]:
%pwd
print(os.getcwd())
# from data_splitter import generate_dataset

/Users/jordanwilke/i3_speech_emotion_recognition


In [52]:
# sys.path.insert(0, './utils/')
from utility_funcs import apply_func_to_all_wavs



# vc = TTS("voice_conversion_models/multilingual/vctk/freevc24", gpu=torch.cuda.is_available())
target_sr = 24000

def create_realistic_voice_clones(input_file, output_file):
  audio, source_sr = torchaudio.load(input_file)
  resampler = torchaudio.transforms.Resample(source_sr, target_sr)
  audio = resampler(audio).squeeze().numpy()

  audio, index = librosa.effects.trim(audio, top_db=30)
  audio = audio / np.max(np.abs(audio))
  audio.dtype = np.float32
  source = preprocess(audio).to(device)
  converted = conversion(source, input_file, reference_embeddings)
  # print(converted)
  torchaudio.save(output_file, torch.from_numpy(converted[input_file]).unsqueeze(0), target_sr)

apply_func_to_all_wavs(input_path, output_path, create_realistic_voice_clones)

  1%|▊                                                                                                 | 58/6903 [01:02<2:02:05,  1.07s/it]


KeyboardInterrupt: 

In [33]:


# import IPython.display as ipd
# for converted_sample, wave2 in converted_dicts:
#   for key, wave in converted_sample.items():
#       print('Converted: %s' % key)
#       display(ipd.Audio(wave, rate=24000))
#       try:
#           print('Reference: %s' % key)
#           display(ipd.Audio(wave2, rate=24000))
#       except:
#           continue

# for key in reference_embeddings:
#   try:
#       print('Original: %s' % key)
#       display(ipd.Audio(reference_embeddings[key][-1], rate=24000))
#   except:
#       continue
# # print('Original:')
# # display(ipd.Audio(audio, rate=24000))



Converted: Data/src/tri-f1-l2.wav


Reference: Data/src/tri-f1-l2.wav


Converted: Data/src/sor-m3-l3.wav


Reference: Data/src/sor-m3-l3.wav


Converted: Data/src/dis-f2-l1.wav


Reference: Data/src/dis-f2-l1.wav


Original: neu-m2-l4.wav
